In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

In [8]:
def teleport_superposition():
    qc = QuantumCircuit(3, 3) # Create 3 qbit
    # Step 1: Alice prepares |+⟩ superposition state
    qc.h(0)  # This creates |+⟩ = (|0⟩ + |1⟩)/√2 On qbit 0
    qc.barrier(label="Alice's |+⟩")

    # Step 2: Create Bell pair (Entanglement qbit 1 and 2)
    qc.h(1)
    qc.cx(1, 2)
    qc.barrier(label="Bell Pair")
    
    # Step 3: Alice's Bell measurement 
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, 0)
    qc.measure(1, 1)
    qc.barrier(label="Alice Measures")
    
    # Step 4: Bob's corrections 
    qc.cx(1, 2)  # X correction 
    qc.cz(0, 2)  # Z correction 
    qc.measure(2, 2)
    
    return qc

# Print circuit diagram
circuit = teleport_superposition()
print("\n Circuit Diagram:")
print(circuit.draw())

# Run simulation
simulator = AerSimulator()
job = simulator.run(circuit, shots=1000) #Run for 1000 shot
result = job.result()
counts = result.get_counts()

# Analyze
print("\nResult Analysis:")
corrected_results = {'0': 0, '1': 0}

for outcome, count in counts.items():
    bob_final_bit = outcome[0] # corrected bit
    alice_bit_1 = outcome[1]   # X-correction bit
    alice_bit_0 = outcome[2]   # Z-correction bit

    # 1. Determine the correct correction string based on Alice's bits
    correction_string = "None"
    if alice_bit_0 == '1' and alice_bit_1 == '0':
        correction_string = "Z-correction"
    elif alice_bit_0 == '0' and alice_bit_1 == '1':
        correction_string = "X-correction"
    elif alice_bit_0 == '1' and alice_bit_1 == '1':
        correction_string = "X and Z-correction"
    
    # 2. Print what actually happened in the circuit
    print(f"  - Alice measured: {alice_bit_0}{alice_bit_1} -> Bob's correction: {correction_string:18} -> Bob's final bit: {bob_final_bit}  (Count: {count})")
    
    # 3. Tally the final, corrected result
    corrected_results[bob_final_bit] += count

# --- This part is now correct ---
print(f"\nBob's Final Result (from q_2):")
total_shots = corrected_results.get('0', 0) + corrected_results.get('1', 0)

print(f"  |0⟩: {corrected_results.get('0', 0)} times ({(corrected_results.get('0', 0) / total_shots * 100):.1f}%)")
print(f"  |1⟩: {corrected_results.get('1', 0)} times ({(corrected_results.get('1', 0) / total_shots * 100):.1f}%)")

superposition_results = corrected_results
print(f"Summary: Bob got {corrected_results}")


 Circuit Diagram:
     ┌───┐ Alice's |+⟩            Bell Pair      ┌───┐┌─┐ Alice Measures      »
q_0: ┤ H ├──────░─────────────────────░───────■──┤ H ├┤M├───────░─────────────»
     └───┘      ░      ┌───┐          ░     ┌─┴─┐└┬─┬┘└╥┘       ░             »
q_1: ───────────░──────┤ H ├──■───────░─────┤ X ├─┤M├──╫────────░──────────■──»
                ░      └───┘┌─┴─┐     ░     └───┘ └╥┘  ║        ░        ┌─┴─┐»
q_2: ───────────░───────────┤ X ├─────░────────────╫───╫────────░────────┤ X ├»
                ░           └───┘     ░            ║   ║        ░        └───┘»
c: 3/══════════════════════════════════════════════╩═══╩══════════════════════»
                                                   1   0                      »
«           
«q_0: ─■────
«      │    
«q_1: ─┼────
«      │ ┌─┐
«q_2: ─■─┤M├
«        └╥┘
«c: 3/════╩═
«         2 

Result Analysis:
  - Alice measured: 10 -> Bob's correction: Z-correction       -> Bob's final bit: 1  (Count: 128)
  - Alice measured: 01 -> B

Next step : trying to prove if the output is really |+> (might have to flip measurement from z-basis to x-basis) 

In [ ]:
from qiskit_aer import AerSimulator
def teleport_superposition(state='plus', measure_basis='Z'):

    qc = QuantumCircuit(3, 3)
    
    # Step 1: Alice prepares the state to teleport
    if state == 'plus':
        qc.h(0)  # |+⟩ = (|0⟩ + |1⟩)/√2
        qc.barrier(label="Alice's |+⟩")
    elif state == 'minus':
        qc.x(0)  # First flip to |1⟩
        qc.h(0)  # |-⟩ = (|0⟩ - |1⟩)/√2
        qc.barrier(label="Alice's |-⟩")
    
    # Step 2: Create Bell pair (entangle qubits 1 and 2)
    qc.h(1)
    qc.cx(1, 2)
    qc.barrier(label="Bell Pair")
    
    # Step 3: Alice's Bell measurement
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, 0)
    qc.measure(1, 1)
    qc.barrier(label="Alice Measures")
    
    # Step 4: Bob's corrections
    qc.cx(1, 2)  # X correction if alice_1 = 1
    qc.cz(0, 2)  # Z correction if alice_0 = 1
    
    # Step 5: Bob measures in chosen basis
    if measure_basis == 'X':
        qc.h(2)  # Convert to X-basis before measurement
        qc.barrier(label="X-basis measure")
    
    qc.measure(2, 2)
    
    return qc

def analyze_results(counts, state_type, measure_basis):
    """Analyze and display results"""
    print(f"\n{'='*70}")
    print(f"State: |{'+' if state_type == 'plus' else '-'}⟩, Measurement Basis: {measure_basis}")
    print(f"{'='*70}")
    
    bob_results = {'0': 0, '1': 0}
    
    for outcome, count in counts.items():
        bob_measured = int(outcome[0])  # What Bob's detector sees
        alice_1 = int(outcome[1])
        alice_0 = int(outcome[2])
        
        # The corrections are ALREADY applied in the circuit before measurement
        # So bob_measured is already the corrected value
        bob_results[str(bob_measured)] += count
        
        correction = "X" if alice_1 == 1 else "-"
        z_correction = "Z" if alice_0 == 1 else "-"
        print(f"  {outcome}: Alice({alice_0}{alice_1}) → [{correction},{z_correction}] → "
              f"Bob measures |{bob_measured}⟩ ({count}×)")
    
    if measure_basis == 'Z':
        print(f"\nBob's Z-basis measurements:")
        print(f"  |0⟩: {bob_results['0']} times ({bob_results['0']/10:.1f}%)")
        print(f"  |1⟩: {bob_results['1']} times ({bob_results['1']/10:.1f}%)")
        print(f"  → Expected: ~50/50 split for superposition states ✓")
        
    else:  # X-basis
        print(f"\nBob's X-basis measurements:")
        print(f"  |+⟩ (reads as 0): {bob_results['0']} times ({bob_results['0']/10:.1f}%)")
        print(f"  |-⟩ (reads as 1): {bob_results['1']} times ({bob_results['1']/10:.1f}%)")
        
        if state_type == 'plus':
            expected = "|+⟩ → ~100% |0⟩"
            success = bob_results['0'] > 900  # Should be close to 100%
        else:
            expected = "|-⟩ → ~100% |1⟩"
            success = bob_results['1'] > 900
        
        status = "✓" if success else "✗ FAILED"
        print(f"  → Expected: {expected} {status}")
    
    return bob_results

# Run all experiments
print("QUANTUM TELEPORTATION VERIFICATION")
simulator = AerSimulator()
all_results = {}

# Experiment 1: |+⟩ measured in Z-basis (should be 50/50)
print("\n[1] Teleporting |+⟩ and measuring in Z-basis (computational basis)")
circuit1 = teleport_superposition(state='plus', measure_basis='Z')
job1 = simulator.run(circuit1, shots=1000)
counts1 = job1.result().get_counts()
all_results['plus_Z'] = analyze_results(counts1, 'plus', 'Z')

# Experiment 2: |+⟩ measured in X-basis (should be all |+⟩ = all 0s)
print("\n[2] Teleporting |+⟩ and measuring in X-basis (superposition basis)")
circuit2 = teleport_superposition(state='plus', measure_basis='X')
job2 = simulator.run(circuit2, shots=1000)
counts2 = job2.result().get_counts()
all_results['plus_X'] = analyze_results(counts2, 'plus', 'X')

# Experiment 3: |-⟩ measured in Z-basis (should be 50/50)
print("\n[3] Teleporting |-⟩ and measuring in Z-basis (computational basis)")
circuit3 = teleport_superposition(state='minus', measure_basis='Z')
job3 = simulator.run(circuit3, shots=1000)
counts3 = job3.result().get_counts()
all_results['minus_Z'] = analyze_results(counts3, 'minus', 'Z')
# Experiment 4: |-⟩ measured in X-basis (should be all |-⟩ = all 1s)
print("\n[4] Teleporting |-⟩ and measuring in X-basis (superposition basis)")
circuit4 = teleport_superposition(state='minus', measure_basis='X')
job4 = simulator.run(circuit4, shots=1000)
counts4 = job4.result().get_counts()
all_results['minus_X'] = analyze_results(counts4, 'minus', 'X')
# Summary
print("VERIFICATION SUMMARY")

plus_x_success = all_results['plus_X']['0'] > 900
minus_x_success = all_results['minus_X']['1'] > 900
print("  • Z-basis measurements: Both |+⟩ and |-⟩ show ~50/50 split")
print("  • X-basis measurements:")
print(f"    - |+⟩ → {all_results['plus_X']['0']/10:.1f}% measured as |+⟩ (expected: ~100%)")
print(f"    - |-⟩ → {all_results['minus_X']['1']/10:.1f}% measured as |-⟩ (expected: ~100%)")
print("\nThis proves the quantum state was truly teleported, not just classical info!")
print("The phase information (the minus sign in |-⟩) survived teleportation!")

# Show circuit diagram
print("Example Circuit: Teleporting |+⟩ with X-basis measurement")
print(circuit2.draw())

QUANTUM TELEPORTATION VERIFICATION

[1] Teleporting |+⟩ and measuring in Z-basis (computational basis)

State: |+⟩, Measurement Basis: Z
  111: Alice(11) → [X,Z] → Bob measures |1⟩ (133×)
  110: Alice(01) → [X,-] → Bob measures |1⟩ (128×)
  010: Alice(01) → [X,-] → Bob measures |0⟩ (113×)
  000: Alice(00) → [-,-] → Bob measures |0⟩ (114×)
  011: Alice(11) → [X,Z] → Bob measures |0⟩ (130×)
  001: Alice(10) → [-,Z] → Bob measures |0⟩ (122×)
  101: Alice(10) → [-,Z] → Bob measures |1⟩ (118×)
  100: Alice(00) → [-,-] → Bob measures |1⟩ (142×)

Bob's Z-basis measurements:
  |0⟩: 479 times (47.9%)
  |1⟩: 521 times (52.1%)
  → Expected: ~50/50 split for superposition states ✓

[2] Teleporting |+⟩ and measuring in X-basis (superposition basis)

State: |+⟩, Measurement Basis: X
  000: Alice(00) → [-,-] → Bob measures |0⟩ (244×)
  011: Alice(11) → [X,Z] → Bob measures |0⟩ (236×)
  001: Alice(10) → [-,Z] → Bob measures |0⟩ (249×)
  010: Alice(01) → [X,-] → Bob measures |0⟩ (271×)

Bob's X-basis m

In [12]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

def teleport_superposition(state='plus', measure_basis='Z'):
    """
    Creates a quantum circuit to teleport a superposition state (|+) or (|-)).
    """
    qc = QuantumCircuit(3, 3) # 3 qubits, 3 classical bits
    
    # Step 1: Alice prepares the state to teleport on qbit 0
    if state == 'plus':
        qc.h(0)  # |+⟩ = (|0⟩ + |1⟩)/√2
        qc.barrier(label="Alice's |+⟩")
    elif state == 'minus':
        qc.x(0)  # First flip to |1⟩
        qc.h(0)  # Then H gate creates |-⟩ = (|0⟩ - |1⟩)/√2
        qc.barrier(label="Alice's |-⟩")
    
    # Step 2: Create Bell pair (entangle qubits 1 and 2)
    qc.h(1)
    qc.cx(1, 2)
    qc.barrier(label="Bell Pair")
    
    # Step 3: Alice's Bell measurement
    qc.cx(0, 1) # CNOT(q_0, q_1)
    qc.h(0)     # H(q_0)
    qc.measure(0, 0) # Measure q_0 -> classical bit c_0
    qc.measure(1, 1) # Measure q_1 -> classical bit c_1
    qc.barrier(label="Alice Measures")
    
    # Step 4: Bob's corrections on qbit 2
    qc.cx(1, 2)  # X correction if c_1 = 1
    qc.cz(0, 2)  # Z correction if c_0 = 1
    
    # Step 5: Bob measures in chosen basis to verify
    if measure_basis == 'X':
        qc.h(2)  # Convert from X-basis to Z-basis before measurement
        qc.barrier(label="X-basis measure")
    
    qc.measure(2, 2) # Measure q_2 -> classical bit c_2
    
    return qc

def analyze_results(counts, state_type, measure_basis):
    """
    Analyzes and displays the results of a teleportation experiment.
    """
    print(f"\nAnalyzing: |{'+' if state_type == 'plus' else '-'}⟩ state, {measure_basis}-basis measurement")
    
    # Qiskit's bit order is reversed: 'c2 c1 c0'
    bob_results = {'0': 0, '1': 0}
    
    for outcome, count in counts.items():
        bob_measured = int(outcome[0])  # Bob's final bit (c_2)
        alice_1 = int(outcome[1])       # Alice's X-correction bit (c_1)
        alice_0 = int(outcome[2])       # Alice's Z-correction bit (c_0)
        
        bob_results[str(bob_measured)] += count
        # Determine the correction string for printing
        correction = "X" if alice_1 == 1 else "-"
        z_correction = "Z" if alice_0 == 1 else "-"
        print(f"  {outcome}: Alice({alice_0}{alice_1}) → [{correction},{z_correction}] → Bob measures |{bob_measured}⟩ ({count}×)")
        
    return bob_results

print("QUANTUM TELEPORTATION VERIFICATION")
simulator = AerSimulator()
all_results = {}

# Experiment 1: |+⟩ measured in Z-basis (should be 50/50)
circuit1 = teleport_superposition(state='plus', measure_basis='Z')
job1 = simulator.run(circuit1, shots=1000)
counts1 = job1.result().get_counts()
all_results['plus_Z'] = analyze_results(counts1, 'plus', 'Z')

# Experiment 2: |+⟩ measured in X-basis (should be all |+⟩ = all 0s)
circuit2 = teleport_superposition(state='plus', measure_basis='X')
job2 = simulator.run(circuit2, shots=1000)
counts2 = job2.result().get_counts()
all_results['plus_X'] = analyze_results(counts2, 'plus', 'X')

# Experiment 3: |-⟩ measured in Z-basis (should be 50/50)
circuit3 = teleport_superposition(state='minus', measure_basis='Z')
job3 = simulator.run(circuit3, shots=1000)
counts3 = job3.result().get_counts()
all_results['minus_Z'] = analyze_results(counts3, 'minus', 'Z')

# Experiment 4: |-⟩ measured in X-basis (should be all |-⟩ = all 1s)
circuit4 = teleport_superposition(state='minus', measure_basis='X')
job4 = simulator.run(circuit4, shots=1000)
counts4 = job4.result().get_counts()
all_results['minus_X'] = analyze_results(counts4, 'minus', 'X')

# SUMMARY 

print("\nVERIFICATION SUMMARY:")

# Get the key success metric
plus_x_success = all_results['plus_X'].get('0', 0) > 900 
minus_x_success = all_results['minus_X'].get('1', 0) > 900
all_passed = plus_x_success and minus_x_success

# Z-Basis Results
print("\nZ-Basis Measurement (Proves superposition):")
print(f"  |+⟩ teleported: {all_results['plus_Z'].get('0', 0)/10:.1f}% |0⟩, {all_results['plus_Z'].get('1', 0)/10:.1f}% |1⟩ (Expected ~50/50)")
print(f"  |-⟩ teleported: {all_results['minus_Z'].get('0', 0)/10:.1f}% |0⟩, {all_results['minus_Z'].get('1', 0)/10:.1f}% |1⟩ (Expected ~50/50)")

# X-Basis Results
print("\nX-Basis Measurement (Proves correct state & phase):")
print(f"  |+⟩ teleported: {all_results['plus_X'].get('0', 0)/10:.1f}% measured as |+⟩. (Expected 100% → {'PASS' if plus_x_success else 'FAIL'})")
print(f"  |-⟩ teleported: {all_results['minus_X'].get('1', 0)/10:.1f}% measured as |-⟩. (Expected 100% → {'PASS' if minus_x_success else 'FAIL'})")

# Circuit diagram
print("\nExample Circuit: Teleporting |+⟩ with X-basis measurement")
print(circuit2.draw())

QUANTUM TELEPORTATION VERIFICATION

Analyzing: |+⟩ state, Z-basis measurement
  010: Alice(01) → [X,-] → Bob measures |0⟩ (140×)
  000: Alice(00) → [-,-] → Bob measures |0⟩ (128×)
  110: Alice(01) → [X,-] → Bob measures |1⟩ (119×)
  011: Alice(11) → [X,Z] → Bob measures |0⟩ (118×)
  100: Alice(00) → [-,-] → Bob measures |1⟩ (125×)
  101: Alice(10) → [-,Z] → Bob measures |1⟩ (124×)
  001: Alice(10) → [-,Z] → Bob measures |0⟩ (126×)
  111: Alice(11) → [X,Z] → Bob measures |1⟩ (120×)

Analyzing: |+⟩ state, X-basis measurement
  001: Alice(10) → [-,Z] → Bob measures |0⟩ (243×)
  000: Alice(00) → [-,-] → Bob measures |0⟩ (241×)
  011: Alice(11) → [X,Z] → Bob measures |0⟩ (273×)
  010: Alice(01) → [X,-] → Bob measures |0⟩ (243×)

Analyzing: |-⟩ state, Z-basis measurement
  111: Alice(11) → [X,Z] → Bob measures |1⟩ (114×)
  011: Alice(11) → [X,Z] → Bob measures |0⟩ (127×)
  010: Alice(01) → [X,-] → Bob measures |0⟩ (125×)
  100: Alice(00) → [-,-] → Bob measures |1⟩ (114×)
  110: Alice(01) → [